# Churn Modeling using Neural Networks and Hyperparameter Tuning

## Introduction

Business problem: A bank has a problem with customer turn-over rates.

Dataset: We have a log of 10,000 customers with their information such as credit score, gender, age, balance, and a yes/no if they exited the bank or not.

Solution: Utilize Neural Networks and Hyperparameter Tuning methods to develop a churn rate modeler which will predict whether a customer will leave the bank or not.

## Data Visualization

In [156]:
# Importing packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')

In [157]:
#Visualizing first few rows
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [158]:
# Getting relevant features for correlation map
corr_map = dataset.iloc[:, 3:].corr()
corr_map[['Exited']].sort_values(by='Exited', ascending=False)

,Exited
Exited,1.000000
Age,0.285323
Balance,0.118533
EstimatedSalary,0.012097
HasCrCard,-0.007138
Tenure,-0.014001
CreditScore,-0.027094
NumOfProducts,-0.047820
IsActiveMember,-0.156128


## Data Preprocessing

First, we import our 10,000 line csv file into a 'dataset' variable. We then split the feature variables into an 'X' variable and whether they exited or not in a 'y' variable. We skip the first 3 columns for the 'X' dataset because row number, customer id, and surname is not relevant for this study.

In [160]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

Here, we check on the first row of our 'X' dataset.

In [161]:
# Visualizing 'X'
X[0]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88], dtype=object)

We have 2 categorical variables that we need to change to numbers. We use scikit-learn's encoders to change it into numbers.

In [162]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [163]:
# After categorical data encoding
X[0]

array([  0.00000000e+00,   0.00000000e+00,   6.19000000e+02,
         0.00000000e+00,   4.20000000e+01,   2.00000000e+00,
         0.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.01348880e+05])

Now we split the datasets into traning and test set, 80% of our data will be used to train our model and 20% will be used to test our model.

In [164]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Since we will be using intensive neural networks, we will scale our features to make it easier on our deep learning processing stages.

In [165]:
# First row of training set before feature scaling
X_train[0]

array([  0.00000000e+00,   1.00000000e+00,   6.67000000e+02,
         0.00000000e+00,   3.40000000e+01,   5.00000000e+00,
         0.00000000e+00,   2.00000000e+00,   1.00000000e+00,
         0.00000000e+00,   1.63830640e+05])

In [166]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [167]:
# First row of training set after feature scaling
X_train[0]

array([-0.5698444 ,  1.74309049,  0.16958176, -1.09168714, -0.46460796,
        0.00666099, -1.21571749,  0.8095029 ,  0.64259497, -1.03227043,
        1.10643166])

## Developing the Neural Network

We start development of our neural network with TensorFlow and Keras. The Sequential package will be used to initalize our neural network. The Dense package will be used to layer our neural network.

In [177]:
# Importing TensorFlow and the Keras libraries with packages
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense

Time to initialize our neural network!

In [178]:
# Initialising the Neural Network
classifier = Sequential()

We add our first layer which takes 11 inputs and has 6 neurons. Generally, we pick a neuron count which is averaged between input (11) and output (1) counts. We use a rectifier(general) activation function for our middle layers and a sigmoid (probablity) function for our final output.

In [179]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

To compile our neural network, we use an 'Adam' optimizer, an excellent algorithm for first-order gradient-based optimization of stochastic objective functions, based on adaptive estimates of lower-order moments. We use a log loss function, binary crossentropy. since it works well with our sigmoid function. Our metrics will be accuracy since that is what we are most concerned about for our model.

In [180]:
# Compiling the Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

To fit our neural network to our training set, we use a batch size of 10 and 100 epochs.

In [181]:
# Fitting the Neural Network to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 4s - loss: 0.4886 - acc: 0.7960     
Epoch 2/100
8000/8000 [==============================] - 2s - loss: 0.4284 - acc: 0.7960     
Epoch 3/100
8000/8000 [==============================] - 2s - loss: 0.4214 - acc: 0.8100     
Epoch 4/100
8000/8000 [==============================] - 1s - loss: 0.4151 - acc: 0.8279     
Epoch 5/100
8000/8000 [==============================] - 2s - loss: 0.4104 - acc: 0.8306     
Epoch 6/100
8000/8000 [==============================] - 1s - loss: 0.4067 - acc: 0.8332     
Epoch 7/100
8000/8000 [==============================] - 1s - loss: 0.4038 - acc: 0.8332     
Epoch 8/100
8000/8000 [==============================] - 1s - loss: 0.4020 - acc: 0.8344     
Epoch 9/100
8000/8000 [==============================] - 1s - loss: 0.4007 - acc: 0.8346     
Epoch 10/100
8000/8000 [==============================] - 2s - loss: 0.3998 - acc: 0.8365     
Epoch 11/100
8000/8000 [==============================] - 2

8000/8000 [==============================] - 2s - loss: 0.3913 - acc: 0.8371     
Epoch 88/100
8000/8000 [==============================] - 2s - loss: 0.3903 - acc: 0.8396     
Epoch 89/100
8000/8000 [==============================] - 2s - loss: 0.3897 - acc: 0.8391     
Epoch 90/100
8000/8000 [==============================] - 2s - loss: 0.3885 - acc: 0.8405     
Epoch 91/100
8000/8000 [==============================] - 2s - loss: 0.3870 - acc: 0.8385     
Epoch 92/100
8000/8000 [==============================] - 2s - loss: 0.3855 - acc: 0.8367     
Epoch 93/100
8000/8000 [==============================] - 2s - loss: 0.3831 - acc: 0.8384     
Epoch 94/100
8000/8000 [==============================] - 2s - loss: 0.3804 - acc: 0.8415     
Epoch 95/100
8000/8000 [==============================] - 2s - loss: 0.3793 - acc: 0.8412     
Epoch 96/100
8000/8000 [==============================] - 2s - loss: 0.3787 - acc: 0.8405     
Epoch 97/100
8000/8000 [==============================] - 2s - 

## Making predictions and evaluating the model

Since we are predicting churn rate, we will use above 50% to be 'True'. We should use a higher threshold for more sensitive studies such as medical ones. After, we print a confusion matrix to see our results.

In [182]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1529   66]
 [ 248  157]]


In [183]:
# Visualizing confusion matrix
pos, neg = cm
tp, fp = pos
fn, tn = neg
print('True Positives:', tp)
print('True Negatives:', tn)
print('False Positives:', fp)
print('False Negatives:', fn)

True Positives: 1529
True Negatives: 157
False Positives: 66
False Negatives: 248


In [184]:
# Accuracy rate of confusion matrix
print('Accuracy:', (tp+tn)/(tp+tn+fp+fn))

Accuracy: 0.843


### Test Neural Network Model

Here we test our neural network with the following data:
* Geograph: France
* Credit score: 650
* Gender: Female
* Age: 20 years old
* Tenure: 2 years
* Balance: 27000 dollars
* Number of products: 2
* Credit card: Yes
* Active member: Yes
* Estimated salary: 50000 dollars

In [20]:
# Making prediction for given values
new_prediction = classifier.predict(
    sc.transform(np.array(
        [[0.0, 0, 650, 0, 20, 2, 27000, 2, 1, 1, 50000]])))
new_prediction_tf = (new_prediction > 0.5)
print('Customer will leave:', new_prediction_tf, new_prediction)

Customer will leave: [[False]] [[ 0.07463048]]


## Evaluating and Tuning the Neural Network

### Evaluating the Neural Network

What is new now is that we will use k-fold cross validation to improve our test set accuracy. To do this, we use scikit-learn's cross val score. We also use keras.wrappers to wrap scikit learn around keras.

In [172]:
# Importing packages and building new classifer with cross validation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [173]:
# Fitting Neural Network to Training set and getting accuracies
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)

Epoch 1/100
7200/7200 [==============================] - 16s - loss: 0.4904 - acc: 0.7967    
Epoch 2/100
7200/7200 [==============================] - 1s - loss: 0.4288 - acc: 0.7971     
Epoch 3/100
7200/7200 [==============================] - 1s - loss: 0.4223 - acc: 0.7971     
Epoch 4/100
7200/7200 [==============================] - 1s - loss: 0.4189 - acc: 0.8153     
Epoch 5/100
7200/7200 [==============================] - 1s - loss: 0.4159 - acc: 0.8260     
Epoch 6/100
7200/7200 [==============================] - 1s - loss: 0.4142 - acc: 0.8294     
Epoch 7/100
7200/7200 [==============================] - 1s - loss: 0.4130 - acc: 0.8296     
Epoch 8/100
7200/7200 [==============================] - 1s - loss: 0.4118 - acc: 0.8324     
Epoch 9/100
7200/7200 [==============================] - 1s - loss: 0.4106 - acc: 0.8318     
Epoch 10/100
7200/7200 [==============================] - 1s - loss: 0.4101 - acc: 0.8317     
Epoch 11/100
7200/7200 [==============================] - 1

7200/7200 [==============================] - 2s - loss: 0.3467 - acc: 0.8617     
Epoch 88/100
7200/7200 [==============================] - 1s - loss: 0.3461 - acc: 0.8615     
Epoch 89/100
7200/7200 [==============================] - 1s - loss: 0.3458 - acc: 0.8604     
Epoch 90/100
7200/7200 [==============================] - 1s - loss: 0.3454 - acc: 0.8624     
Epoch 91/100
7200/7200 [==============================] - 1s - loss: 0.3459 - acc: 0.8626     
Epoch 92/100
7200/7200 [==============================] - 1s - loss: 0.3449 - acc: 0.8625     
Epoch 93/100
7200/7200 [==============================] - 1s - loss: 0.3451 - acc: 0.8619     
Epoch 94/100
7200/7200 [==============================] - 1s - loss: 0.3440 - acc: 0.8610     
Epoch 95/100
7200/7200 [==============================] - 2s - loss: 0.3445 - acc: 0.8610     
Epoch 96/100
7200/7200 [==============================] - 1s - loss: 0.3443 - acc: 0.8610     
Epoch 97/100
7200/7200 [==============================] - 1s - 

7200/7200 [==============================] - 1s - loss: 0.3334 - acc: 0.8635     
Epoch 73/100
7200/7200 [==============================] - 2s - loss: 0.3337 - acc: 0.8653     
Epoch 74/100
7200/7200 [==============================] - 1s - loss: 0.3336 - acc: 0.8649     
Epoch 75/100
7200/7200 [==============================] - 1s - loss: 0.3328 - acc: 0.8636     
Epoch 76/100
7200/7200 [==============================] - 1s - loss: 0.3328 - acc: 0.8651     
Epoch 77/100
7200/7200 [==============================] - 1s - loss: 0.3329 - acc: 0.8653     
Epoch 78/100
7200/7200 [==============================] - 1s - loss: 0.3335 - acc: 0.8661     
Epoch 79/100
7200/7200 [==============================] - 1s - loss: 0.3333 - acc: 0.8657     
Epoch 80/100
7200/7200 [==============================] - 1s - loss: 0.3338 - acc: 0.8651     
Epoch 81/100
7200/7200 [==============================] - 1s - loss: 0.3331 - acc: 0.8644     
Epoch 82/100
7200/7200 [==============================] - 1s - 

7200/7200 [==============================] - 1s - loss: 0.3981 - acc: 0.8344     
Epoch 58/100
7200/7200 [==============================] - 1s - loss: 0.3981 - acc: 0.8362     
Epoch 59/100
7200/7200 [==============================] - 1s - loss: 0.3976 - acc: 0.8376     
Epoch 60/100
7200/7200 [==============================] - 1s - loss: 0.3976 - acc: 0.8343     
Epoch 61/100
7200/7200 [==============================] - 1s - loss: 0.3975 - acc: 0.8362     
Epoch 62/100
7200/7200 [==============================] - 1s - loss: 0.3972 - acc: 0.8375     
Epoch 63/100
7200/7200 [==============================] - 1s - loss: 0.3970 - acc: 0.8389     
Epoch 64/100
7200/7200 [==============================] - 1s - loss: 0.3974 - acc: 0.8369     
Epoch 65/100
7200/7200 [==============================] - 1s - loss: 0.3962 - acc: 0.8357     
Epoch 66/100
7200/7200 [==============================] - 1s - loss: 0.3974 - acc: 0.8381     
Epoch 67/100
7200/7200 [==============================] - 1s - 

7200/7200 [==============================] - 1s - loss: 0.3953 - acc: 0.8382     
Epoch 43/100
7200/7200 [==============================] - 1s - loss: 0.3952 - acc: 0.8400     
Epoch 44/100
7200/7200 [==============================] - 1s - loss: 0.3950 - acc: 0.8385     
Epoch 45/100
7200/7200 [==============================] - 1s - loss: 0.3942 - acc: 0.8374     
Epoch 46/100
7200/7200 [==============================] - 1s - loss: 0.3945 - acc: 0.8392     
Epoch 47/100
7200/7200 [==============================] - 1s - loss: 0.3946 - acc: 0.8385     
Epoch 48/100
7200/7200 [==============================] - 1s - loss: 0.3946 - acc: 0.8367     
Epoch 49/100
7200/7200 [==============================] - 1s - loss: 0.3942 - acc: 0.8383     
Epoch 50/100
7200/7200 [==============================] - 1s - loss: 0.3942 - acc: 0.8390     
Epoch 51/100
7200/7200 [==============================] - 1s - loss: 0.3942 - acc: 0.8375     
Epoch 52/100
7200/7200 [==============================] - 1s - 

...

7200/7200 [==============================] - 1s - loss: 0.3985 - acc: 0.8354     
Epoch 99/100
7200/7200 [==============================] - 1s - loss: 0.3979 - acc: 0.8351     
Epoch 100/100
7200/7200 [==============================] - 3s - loss: 0.4860 - acc: 0.7969     
Epoch 2/100
7200/7200 [==============================] - 2s - loss: 0.4293 - acc: 0.7969     
Epoch 3/100
7200/7200 [==============================] - 1s - loss: 0.4232 - acc: 0.7969     
Epoch 4/100
7200/7200 [==============================] - 1s - loss: 0.4188 - acc: 0.8144     
Epoch 5/100
7200/7200 [==============================] - 1s - loss: 0.4164 - acc: 0.8261     
Epoch 6/100
7200/7200 [==============================] - 1s - loss: 0.4147 - acc: 0.8300     
Epoch 7/100
7200/7200 [==============================] - 2s - loss: 0.4133 - acc: 0.8311     
Epoch 8/100
7200/7200 [==============================] - 2s - loss: 0.4117 - acc: 0.8321     
Epoch 9/100
7200/7200 [==============================] - 1s - loss: 0

7200/7200 [==============================] - 1s - loss: 0.4003 - acc: 0.8357     
Epoch 84/100
7200/7200 [==============================] - 1s - loss: 0.3999 - acc: 0.8367     
Epoch 85/100
7200/7200 [==============================] - 1s - loss: 0.4004 - acc: 0.8356     
Epoch 86/100
7200/7200 [==============================] - 1s - loss: 0.4005 - acc: 0.8343     
Epoch 87/100
7200/7200 [==============================] - 2s - loss: 0.4003 - acc: 0.8347     
Epoch 88/100
7200/7200 [==============================] - 1s - loss: 0.4003 - acc: 0.8343     
Epoch 89/100
7200/7200 [==============================] - 1s - loss: 0.3999 - acc: 0.8349     
Epoch 90/100
7200/7200 [==============================] - 1s - loss: 0.4003 - acc: 0.8356     
Epoch 91/100
7200/7200 [==============================] - 1s - loss: 0.3994 - acc: 0.8364     
Epoch 92/100
7200/7200 [==============================] - 1s - loss: 0.4000 - acc: 0.8331     
Epoch 93/100
7200/7200 [==============================] - 1s - 

7200/7200 [==============================] - 1s - loss: 0.3982 - acc: 0.8365     
Epoch 69/100
7200/7200 [==============================] - 1s - loss: 0.3981 - acc: 0.8362     
Epoch 70/100
7200/7200 [==============================] - 1s - loss: 0.3978 - acc: 0.8376     
Epoch 71/100
7200/7200 [==============================] - 1s - loss: 0.3984 - acc: 0.8360     
Epoch 72/100
7200/7200 [==============================] - 1s - loss: 0.3974 - acc: 0.8372     
Epoch 73/100
7200/7200 [==============================] - 1s - loss: 0.3976 - acc: 0.8360     
Epoch 74/100
7200/7200 [==============================] - 2s - loss: 0.3976 - acc: 0.8374     
Epoch 75/100
7200/7200 [==============================] - 2s - loss: 0.3979 - acc: 0.8360     
Epoch 76/100
7200/7200 [==============================] - 2s - loss: 0.3972 - acc: 0.8365     
Epoch 77/100
7200/7200 [==============================] - 2s - loss: 0.3975 - acc: 0.8367     
Epoch 78/100
7200/7200 [==============================] - 2s - 

7200/7200 [==============================] - 2s - loss: 0.3983 - acc: 0.8364     
Epoch 54/100
7200/7200 [==============================] - 1s - loss: 0.3984 - acc: 0.8374     
Epoch 55/100
7200/7200 [==============================] - 2s - loss: 0.3978 - acc: 0.8356     
Epoch 56/100
7200/7200 [==============================] - 2s - loss: 0.3975 - acc: 0.8371     
Epoch 57/100
7200/7200 [==============================] - 2s - loss: 0.3978 - acc: 0.8371     
Epoch 58/100
7200/7200 [==============================] - 2s - loss: 0.3977 - acc: 0.8360     
Epoch 59/100
7200/7200 [==============================] - 2s - loss: 0.3984 - acc: 0.8374     
Epoch 60/100
7200/7200 [==============================] - 1s - loss: 0.3978 - acc: 0.8354     
Epoch 61/100
7200/7200 [==============================] - 2s - loss: 0.3980 - acc: 0.8365     
Epoch 62/100
7200/7200 [==============================] - 2s - loss: 0.3979 - acc: 0.8367     
Epoch 63/100
7200/7200 [==============================] - 2s - 

7200/7200 [==============================] - 1s - loss: 0.4042 - acc: 0.8318     
Epoch 39/100
7200/7200 [==============================] - 2s - loss: 0.4052 - acc: 0.8337     
Epoch 40/100
7200/7200 [==============================] - 1s - loss: 0.4046 - acc: 0.8344     
Epoch 41/100
7200/7200 [==============================] - 1s - loss: 0.4053 - acc: 0.8322     
Epoch 42/100
7200/7200 [==============================] - 1s - loss: 0.4049 - acc: 0.8336     
Epoch 43/100
7200/7200 [==============================] - 1s - loss: 0.4043 - acc: 0.8340     
Epoch 44/100
7200/7200 [==============================] - 1s - loss: 0.4048 - acc: 0.8337     
Epoch 45/100
7200/7200 [==============================] - 1s - loss: 0.4049 - acc: 0.8325     
Epoch 46/100
7200/7200 [==============================] - 2s - loss: 0.4045 - acc: 0.8343     
Epoch 47/100
7200/7200 [==============================] - 2s - loss: 0.4040 - acc: 0.8335     
Epoch 48/100
7200/7200 [==============================] - 2s - 

Our performance here merely improved by a fraction of a percent. Time to tune our neural network even further.

In [174]:
mean = accuracies.mean()
variance = accuracies.std()
print('Average: ', ('%.4f' % mean))
print('Standard Deviation: ', ('%.4f' % variance))

Average:  0.8350
Standard Deviation:  0.0119


### Tuning the Neural Network

To finally tune our neural network, we use grid search to tune our hyper parameters such as batch size and epochs.

In [146]:
# Importing packages and building new classifer with grid search
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [147]:
# Building our Neural Network with empty optimizer, batch size, and epochs
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)

In [152]:
# Parameters for batch size, epochs, and optimizer functions
parameters = {'batch_size': [16, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}

In [153]:
# Setting up Grid Search
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/100
7200/7200 [==============================] - 1s - loss: 0.5106 - acc: 0.7968     
Epoch 2/100
7200/7200 [==============================] - 0s - loss: 0.4316 - acc: 0.7971     
Epoch 3/100
7200/7200 [==============================] - 0s - loss: 0.4248 - acc: 0.7971     
Epoch 4/100
7200/7200 [==============================] - 0s - loss: 0.4209 - acc: 0.7987     
Epoch 5/100
7200/7200 [==============================] - 0s - loss: 0.4169 - acc: 0.8217     
Epoch 6/100
7200/7200 [==============================] - 0s - loss: 0.4157 - acc: 0.8232     
Epoch 7/100
7200/7200 [==============================] - 0s - loss: 0.4142 - acc: 0.8278     
Epoch 8/100
7200/7200 [==============================] - 0s - loss: 0.4128 - acc: 0.8283     
Epoch 9/100
7200/7200 [==============================] - 0s - loss: 0.4117 - acc: 0.8303     
Epoch 10/100
7200/7200 [==============================] - 0s - loss: 0.4108 - acc: 0.8326     
Epoch 11/100
7200/7200 [==============================] - 0

7200/7200 [==============================] - 0s - loss: 0.3951 - acc: 0.8382     
Epoch 87/100
7200/7200 [==============================] - 0s - loss: 0.3951 - acc: 0.8371     
Epoch 88/100
7200/7200 [==============================] - 0s - loss: 0.3945 - acc: 0.8372     
Epoch 89/100
7200/7200 [==============================] - 0s - loss: 0.3945 - acc: 0.8374     
Epoch 90/100
7200/7200 [==============================] - 0s - loss: 0.3945 - acc: 0.8382     
Epoch 91/100
7200/7200 [==============================] - 0s - loss: 0.3944 - acc: 0.8356     
Epoch 92/100
7200/7200 [==============================] - 0s - loss: 0.3944 - acc: 0.8387     
Epoch 93/100
7200/7200 [==============================] - 0s - loss: 0.3939 - acc: 0.8363     
Epoch 94/100
7200/7200 [==============================] - 0s - loss: 0.3940 - acc: 0.8372     
Epoch 95/100
7200/7200 [==============================] - 0s - loss: 0.3942 - acc: 0.8371     
Epoch 96/100
7200/7200 [==============================] - 0s - 

7200/7200 [==============================] - 0s - loss: 0.3331 - acc: 0.8672     
Epoch 72/100
7200/7200 [==============================] - 0s - loss: 0.3332 - acc: 0.8647     
Epoch 73/100
7200/7200 [==============================] - 0s - loss: 0.3330 - acc: 0.8674     
Epoch 74/100
7200/7200 [==============================] - 0s - loss: 0.3327 - acc: 0.8661     
Epoch 75/100
7200/7200 [==============================] - 0s - loss: 0.3323 - acc: 0.8672     
Epoch 76/100
7200/7200 [==============================] - 0s - loss: 0.3329 - acc: 0.8654     
Epoch 77/100
7200/7200 [==============================] - 0s - loss: 0.3325 - acc: 0.8656     
Epoch 78/100
7200/7200 [==============================] - 0s - loss: 0.3330 - acc: 0.8650     
Epoch 79/100
7200/7200 [==============================] - 0s - loss: 0.3329 - acc: 0.8665     
Epoch 80/100
7200/7200 [==============================] - 0s - loss: 0.3323 - acc: 0.8664     
Epoch 81/100
7200/7200 [==============================] - 0s - 

7200/7200 [==============================] - 0s - loss: 0.3426 - acc: 0.8567     
Epoch 56/100
7200/7200 [==============================] - 0s - loss: 0.3424 - acc: 0.8611     
Epoch 57/100
7200/7200 [==============================] - 0s - loss: 0.3430 - acc: 0.8611     
Epoch 58/100
7200/7200 [==============================] - ETA: 0s - loss: 0.3436 - acc: 0.859 - 0s - loss: 0.3428 - acc: 0.8597     
Epoch 59/100
7200/7200 [==============================] - 0s - loss: 0.3420 - acc: 0.8606     
Epoch 60/100
7200/7200 [==============================] - 0s - loss: 0.3419 - acc: 0.8597     
Epoch 61/100
7200/7200 [==============================] - 0s - loss: 0.3431 - acc: 0.8576     
Epoch 62/100
7200/7200 [==============================] - 0s - loss: 0.3422 - acc: 0.8588     
Epoch 63/100
7200/7200 [==============================] - 0s - loss: 0.3412 - acc: 0.8582     
Epoch 64/100
7200/7200 [==============================] - 0s - loss: 0.3411 - acc: 0.8590     
Epoch 65/100
7200/7200 [=

7200/7200 [==============================] - 0s - loss: 0.3454 - acc: 0.8619     - ETA: 0s - loss: 0.3395 - acc: 0.
Epoch 41/100
7200/7200 [==============================] - 0s - loss: 0.3450 - acc: 0.8603     
Epoch 42/100
7200/7200 [==============================] - 0s - loss: 0.3449 - acc: 0.8607     - ETA: 0s - loss: 0.3486 - acc: 
Epoch 43/100
7200/7200 [==============================] - 0s - loss: 0.3452 - acc: 0.8607     
Epoch 44/100
7200/7200 [==============================] - 0s - loss: 0.3452 - acc: 0.8617     
Epoch 45/100
7200/7200 [==============================] - 0s - loss: 0.3446 - acc: 0.8624     
Epoch 46/100
7200/7200 [==============================] - 0s - loss: 0.3437 - acc: 0.8608     
Epoch 47/100
7200/7200 [==============================] - 0s - loss: 0.3434 - acc: 0.8617     
Epoch 48/100
7200/7200 [==============================] - 0s - loss: 0.3449 - acc: 0.8603     
Epoch 49/100
7200/7200 [==============================] - 0s - loss: 0.3436 - acc: 0.8642   

...

8000/8000 [==============================] - 0s - loss: 0.3993 - acc: 0.8350     
Epoch 107/500
8000/8000 [==============================] - 0s - loss: 0.3995 - acc: 0.8361     
Epoch 108/500
8000/8000 [==============================] - 0s - loss: 0.3994 - acc: 0.8357     
Epoch 109/500
8000/8000 [==============================] - 0s - loss: 0.3995 - acc: 0.8346     
Epoch 110/500
8000/8000 [==============================] - 0s - loss: 0.3996 - acc: 0.8365     
Epoch 111/500
8000/8000 [==============================] - 0s - loss: 0.3995 - acc: 0.8349     
Epoch 112/500
8000/8000 [==============================] - 0s - loss: 0.3992 - acc: 0.8351     
Epoch 113/500
8000/8000 [==============================] - 0s - loss: 0.3994 - acc: 0.8349     
Epoch 114/500
8000/8000 [==============================] - 0s - loss: 0.3993 - acc: 0.8350     
Epoch 115/500
8000/8000 [==============================] - 0s - loss: 0.3995 - acc: 0.8354     
Epoch 116/500
8000/8000 [=============================

8000/8000 [==============================] - 0s - loss: 0.3989 - acc: 0.8367     
Epoch 192/500
8000/8000 [==============================] - 0s - loss: 0.3988 - acc: 0.8349     
Epoch 193/500
8000/8000 [==============================] - 0s - loss: 0.3989 - acc: 0.8360     
Epoch 194/500
8000/8000 [==============================] - 0s - loss: 0.3991 - acc: 0.8361     
Epoch 195/500
8000/8000 [==============================] - 0s - loss: 0.3988 - acc: 0.8364     
Epoch 196/500
8000/8000 [==============================] - 0s - loss: 0.3988 - acc: 0.8361     
Epoch 197/500
8000/8000 [==============================] - 0s - loss: 0.3988 - acc: 0.8361     
Epoch 198/500
8000/8000 [==============================] - 0s - loss: 0.3987 - acc: 0.8366     
Epoch 199/500
8000/8000 [==============================] - 0s - loss: 0.3988 - acc: 0.8354     
Epoch 200/500
8000/8000 [==============================] - 0s - loss: 0.3988 - acc: 0.8363     
Epoch 201/500
8000/8000 [=============================

8000/8000 [==============================] - 0s - loss: 0.3983 - acc: 0.8366     
Epoch 277/500
8000/8000 [==============================] - 0s - loss: 0.3983 - acc: 0.8363     
Epoch 278/500
8000/8000 [==============================] - 0s - loss: 0.3982 - acc: 0.8342     
Epoch 279/500
8000/8000 [==============================] - 0s - loss: 0.3986 - acc: 0.8374     
Epoch 280/500
8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8360     
Epoch 281/500
8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8370     
Epoch 282/500
8000/8000 [==============================] - 0s - loss: 0.3986 - acc: 0.8344     
Epoch 283/500
8000/8000 [==============================] - 0s - loss: 0.3986 - acc: 0.8376     
Epoch 284/500
8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8354     
Epoch 285/500
8000/8000 [==============================] - 0s - loss: 0.3984 - acc: 0.8360     
Epoch 286/500
8000/8000 [=============================

8000/8000 [==============================] - 0s - loss: 0.3983 - acc: 0.8365     
Epoch 362/500
8000/8000 [==============================] - 0s - loss: 0.3983 - acc: 0.8365     
Epoch 363/500
8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8367     
Epoch 364/500
8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8351     
Epoch 365/500
8000/8000 [==============================] - 0s - loss: 0.3982 - acc: 0.8364     
Epoch 366/500
8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8354     
Epoch 367/500
8000/8000 [==============================] - 0s - loss: 0.3982 - acc: 0.8367     
Epoch 368/500
8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8363     
Epoch 369/500
8000/8000 [==============================] - 0s - loss: 0.3984 - acc: 0.8355     
Epoch 370/500
8000/8000 [==============================] - 0s - loss: 0.3983 - acc: 0.8363     
Epoch 371/500
8000/8000 [=============================

8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8351     
Epoch 447/500
8000/8000 [==============================] - 0s - loss: 0.3979 - acc: 0.8346     
Epoch 448/500
8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8369     
Epoch 449/500
8000/8000 [==============================] - 0s - loss: 0.3983 - acc: 0.8371     
Epoch 450/500
8000/8000 [==============================] - 0s - loss: 0.3983 - acc: 0.8355     
Epoch 451/500
8000/8000 [==============================] - 0s - loss: 0.3979 - acc: 0.8363     
Epoch 452/500
8000/8000 [==============================] - 0s - loss: 0.3985 - acc: 0.8357     
Epoch 453/500
8000/8000 [==============================] - 0s - loss: 0.3982 - acc: 0.8356     
Epoch 454/500
8000/8000 [==============================] - 0s - loss: 0.3983 - acc: 0.8356     
Epoch 455/500
8000/8000 [==============================] - 0s - loss: 0.3983 - acc: 0.8360     
Epoch 456/500
8000/8000 [=============================

In [154]:
# Getting our best parameters
best_parameters

{'batch_size': 32, 'epochs': 500, 'optimizer': 'rmsprop'}

In [175]:
# Getting our best average
print('Best average: ', ('%.4f' % best_accuracy))

Best average:  0.8511


## Conclusion

Our original model scored a 84.15%. With hyper parameter tuning, the average was increased by a percent, up to 85.11%.

Possible improvements could be increasing the batch size to 64 and increasing the epoch to 1000 to see if there could be better accuracy rates. However, there would be a need of a GPU for this application. Just this notebook alone took 10 hours to run on an i5-6400 CPU.